# Exploratory Data Analysis - Renewable Energy Data

In [1]:
import pandas as pd
import numpy as np

## Renewable Energy Data

We first load the data from United Nations which contains the renewable energy data. The columns are Country, Energy Supply, Energy Supply per Capita and % Renewable energy. We rename the columns as these. We then perform the following data cleaning:

1. Converting the energy to gigajoule values

2. Rename the following list of countries:

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

3. Rename countries with numbers and/or parenthesis in their name



In [2]:
energy = pd.read_excel('Energy Indicators.xls')

energy = energy[16:243]  ## the actual data is contained within these rows

column_list = energy.columns

energy = energy[column_list[2:]]  #first column dropped

new_name_list = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']

energy.columns = new_name_list  #rename columns

energy.replace('...', np.NaN,inplace = True)     #replace '...' with nan

energy['Energy Supply']*= 100000       #convert energy units to Gigajoule

new_country_names = {"Republic of Korea": "South Korea",
                    "United States of America20": "United States",
                    "United Kingdom of Great Britain and Northern Ireland19":"United Kingdom",
                    "China, Hong Kong Special Administrative Region": "Hong Kong"}

energy.replace({"Country": new_country_names},inplace = True)

energy['Country'] = energy['Country'].str.replace('\d+', '')   # remove numbers from Country names, \d+ stands
                                                               # for any digit
    
energy['Country'] = energy['Country'].str.replace(r"\s+\(.*\)","") #remove expression within brackets

energy['Country'] = energy['Country'].map(lambda x: x.strip())  #remove whitespace

energy.head()

,Country,Energy Supply,Energy Supply per Capita,% Renewable
16,Afghanistan,32100000.0,10.0,78.669280
17,Albania,10200000.0,35.0,100.000000
18,Algeria,195900000.0,51.0,0.551010
19,American Samoa,NaN,NaN,0.641026
20,Andorra,900000.0,121.0,88.695650


## GDP Data from World Bank

It is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD).

We skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

In [3]:
gdp = pd.read_csv('world_bank.csv', skiprows=4)

new_country_names = {"Korea, Rep.": "South Korea",
                    "Iran, Islamic Rep.": "Iran",
                    "Hong Kong SAR, China":"Hong Kong"}

gdp.replace({"Country Name": new_country_names},inplace = True)

gdp.rename(columns={'Country Name': 'Country'}, inplace=True)

gdp.head()

,Country,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,Aruba,ABW,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.467704e+09,NaN,NaN,NaN,NaN,NaN
1,Andorra,AND,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,4.018196e+09,4.021331e+09,3.675728e+09,3.535389e+09,3.346317e+09,3.185605e+09,3.129538e+09,3.127550e+09,NaN,NaN
2,Afghanistan,AFG,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.030523e+10,1.172119e+10,1.214448e+10,1.469733e+10,1.593680e+10,1.691113e+10,1.935220e+10,1.973134e+10,1.999032e+10,2.029415e+10
3,Angola,AGO,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,5.581103e+10,6.842044e+10,7.787420e+10,7.975320e+10,8.247091e+10,8.570262e+10,9.012096e+10,9.626143e+10,1.008863e+11,1.039106e+11
4,Albania,ALB,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,9.771760e+09,1.034829e+10,1.112752e+10,1.150029e+10,1.192695e+10,1.223109e+10,1.240477e+10,1.254247e+10,1.279331e+10,1.312082e+10


## Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology

In [4]:
ScimEn = pd.read_excel('scimagojr-3.xlsx')

ScimEn.head()

,Rank,Country,Documents,Citable documents,Citations,Self-citations,Citations per document,H index
0,1,China,127050,126767,597237,411683,4.70,138
1,2,United States,96661,94747,792274,265436,8.20,230
2,3,Japan,30504,30287,223024,61554,7.31,134
3,4,United Kingdom,20944,20357,206091,37874,9.84,139
4,5,Russian Federation,18534,18301,34266,12422,1.85,57


## Merging the DataFrames. We merge on the country names using the GDP data from 2006 to 2015 and keep only countries with rank 1 to 15 in the Scimago ranking

In [5]:
new_df = pd.merge(energy, gdp, on='Country')

new_df = pd.merge(new_df, ScimEn, on ='Country')

new_df.set_index('Country',inplace=True)       #set Country as the index

columns_to_keep = ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 
                  'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', 
                  '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', 
                 '2013', '2014', '2015']

new_df = new_df[columns_to_keep]

new_df = new_df[new_df['Rank'] <= 15]

new_df.sort_values(by=['Rank'], inplace=True)

new_df

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,127050,126767,597237,411683,4.70,138,1.271910e+10,93.0,19.754910,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
United States,2,96661,94747,792274,265436,8.20,230,9.083800e+09,286.0,11.570980,1.479230e+13,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13
Japan,3,30504,30287,223024,61554,7.31,134,1.898400e+09,149.0,10.232820,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12
United Kingdom,4,20944,20357,206091,37874,9.84,139,7.920000e+08,124.0,10.600470,2.419631e+12,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12
Russian Federation,5,18534,18301,34266,12422,1.85,57,3.070900e+09,214.0,17.288680,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12
Canada,6,17899,17620,215003,40930,12.01,149,1.043100e+09,296.0,61.945430,1.564469e+12,1.596740e+12,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12
Germany,7,17027,16831,140566,27426,8.26,126,1.326100e+09,165.0,17.901530,3.332891e+12,3.441561e+12,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12
India,8,15005,14841,128763,37209,8.58,115,3.319500e+09,26.0,14.969080,1.265894e+12,1.374865e+12,1.428361e+12,1.549483e+12,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12
France,9,13153,12973,130632,28601,9.93,114,1.059700e+09,166.0,17.020280,2.607840e+12,2.669424e+12,2.674637e+12,2.595967e+12,2.646995e+12,2.702032e+12,2.706968e+12,2.722567e+12,2.729632e+12,2.761185e+12


## The average GDP over the last 10 years for each country

In [6]:
def average_gdp(df):
    top_15_data = df.copy()
    column_list = ['2006','2007','2008','2009','2010','2011','2012','2013','2014','2015']
    avg_gdp = top_15_data[column_list].mean(axis=1)
    return avg_gdp.sort_values(ascending=False)

In [7]:
average_gdp(new_df)

Country
United States         1.536434e+13
China                 6.348609e+12
Japan                 5.542208e+12
Germany               3.493025e+12
France                2.681725e+12
United Kingdom        2.487907e+12
Brazil                2.189794e+12
Italy                 2.120175e+12
India                 1.769297e+12
Canada                1.660647e+12
Russian Federation    1.565459e+12
Spain                 1.418078e+12
Australia             1.164043e+12
South Korea           1.106715e+12
Iran                  4.441558e+11
dtype: float64

## To find the GDP change of a particular country

For this we first find the country with the 3rd largest Average GDP and then calculate its GDP change from 2006 to 2015

In [10]:
def gdp_change(df):
    
    gdp_data = df.copy()
    
    gdp_data['average GDP'] = average_gdp(df)  # call the function created in the last cell which now adds a new column
    
    gdp_data.sort_values(by='average GDP', inplace=True, ascending=False) #sort Top15 by averageGDP
    
    max_change = abs(gdp_data['2015'].iloc[2] - gdp_data['2006'].iloc[2])
    
    return max_change
    
    

In [11]:
gdp_change(new_df)

173021625777.66016

## To find the mean energy supply per capita

In [12]:
def mean_energy_per_capita(df):
    energy_per_capita = df.copy()
    
    mean_energy = df['Energy Supply per Capita'].mean(axis=0)
    
    return mean_energy


In [13]:
mean_energy_per_capita(new_df)

157.6

## To find the country with maximum % Renewable the percentage?

In [22]:
def max_renewable(df):
    
    max_renew = df.copy()
    
    max_renew.sort_values(by='% Renewable', inplace=True, ascending=False) #sort Top15 by averageGDP
    
    return (max_renew.iloc[0].name, max_renew['% Renewable'].iloc[0]) # .name returns the index name

    

In [23]:
max_renewable(new_df)

('Brazil', 69.64803)

## Find the country with the maximum Self Citations/Total Citations

For this we first need to create a new column with the self citations/total citations value

In [28]:
def max_cite_ratio(df):
    
    cite = df.copy()
    
    cite['Cite Ratio'] = cite['Self-citations']/cite['Citations']
    
    max_cite_country = cite['Cite Ratio'].idxmax(axis=0)
    
    return (max_cite_country, cite['Cite Ratio'].loc[max_cite_country])

In [29]:
max_cite_ratio(new_df)

('China', 0.6893126179389422)

## Finding the second most populous country

For this we first estimate the total population from the energy supply and the energy supply per capita data. We then extract the data for the second most populous country

In [32]:
def population(df):
    
    population = df.copy()
    
    population['Total Population'] = population['Energy Supply']/population['Energy Supply per Capita']
    
    population.sort_values(by='Total Population', inplace=True, ascending=False) #sort Top15 by averageGDP
    
    return (population.iloc[1].name, population['Total Population'].iloc[1]) # .name returns the index name
    
    

In [33]:
population(new_df)

('India', 127673076.92307693)

## Find out which countries have more than the median % Renewable energy

In [36]:
def median_mask(df):
    
    Top15 = df.copy()
    
    median_all_country = Top15['% Renewable'].median()
    
    Top15['HighRenew'] = Top15['% Renewable'].apply(lambda x: 0 if x < median_all_country else 1)
    
    Top15.sort_values(by='Rank', inplace=True)
    
    return Top15['HighRenew']



In [37]:
median_mask(new_df)

Country
China                 1
United States         0
Japan                 0
United Kingdom        0
Russian Federation    1
Canada                1
Germany               1
India                 0
France                1
South Korea           0
Italy                 1
Spain                 1
Iran                  0
Australia             0
Brazil                1
Name: HighRenew, dtype: int64